In [1]:
#Uses python3

import numpy as np
import time
import pandas as pd
import sys

In [17]:
#====================== all functions ======================
def coord(site):
    """get coordinate i of vector"""
    x = site // L
    y = site - x*L
    return (x,y)

def get(i):
    """fixin' boundary"""
    if i<0: return i
    else: return i % L
    
def get_neigh():
    """get neighbour's arr"""
    s = np.arange(L**2).reshape(L,L)
    nei = []
    for site in range(L*L):
        i,j = coord(site)
        nei += [s[get(i-1),get(j)],s[get(i),get(j+1)],s[get(i+1),get(j)],s[get(i),get(j-1)]]
    return np.array(nei, dtype=np.int32).reshape(L*L,4)

#################################################################
def gen_state():
    """generate random start state with lenght L*L and [-1,1] components"""
    state_s = np.array([np.random.choice([-1,1]) for _ in range(L*L)], dtype=np.int32)
    state_t = np.array([np.random.choice([-1,1]) for _ in range(L*L)], dtype=np.int32)
    return state_s, state_t

def mc_choice(dE,T):
    """принимаем или не принимаем переворот спина?"""
    if dE < 0:
        return True
    elif np.random.uniform() <= np.exp(-dE/T):
        return True
    else:
        return False 
    
def calc_dE(site_1, site_2, neigh_1, neigh_2):
    """calculate dE"""
    de = 0
    for i in range(4):
        de += 2*site_1*(site_2*neigh_1[i]*neigh_2[i]+neigh_1[i])
    return de

def step(s,t,nei,T):
    """крутим 1 спин"""
    j,k = np.random.randint(-1,L-1), np.random.randint(-1,L-1) 
    if j==-1: j=L-1
    if k==-1: k=L-1
    site = j*L+k
    
    if np.random.choice([-1,1])==1:
        neigh_s = s[nei[site,:]]  
        neigh_t = t[nei[site,:]]  
        dE = calc_dE(s[site], t[site], neigh_s, neigh_t)   # if lattice s -> pass s,t
        if mc_choice(dE,T):
            s[site] *= -1
    else: 
        neigh_s = s[nei[site,:]]  
        neigh_t = t[nei[site,:]]  
        dE = calc_dE(t[site], s[site], neigh_t, neigh_s)   # if lattice t -> pass t,s
        if mc_choice(dE,T):
            t[site] *= -1

# def mc_step(s, t, nei, T):
#     """perform 2*L*L steps MC to relax"""
#     for _ in range(2*L*L):
#         step(s,t,nei,T)
        
#################################################################
def expand(state):
    _a = np.concatenate((state[L-1].reshape(1,L), state), axis=0)   # add last line to TOP
    a = np.concatenate((_a, _a[:,0].reshape(L+1,1)), axis=1)    # add first line to RIGHT
    return a

def calc_e(s,t):
    s,t = s.reshape(L,L),t.reshape(L,L)
    s,t = expand(s), expand(t)
    e = 0
    for i in range(L):
        for j in range(L):
            e += s[i,j]*s[i+1,j] + t[i,j]*t[i+1,j] + s[i,j]*s[i+1,j]*t[i,j]*t[i+1,j] # right neighbour
            e += s[i,j]*s[i,j+1] + t[i,j]*t[i,j+1] + s[i,j]*s[i,j+1]*t[i,j]*t[i,j+1] # down neighbour
    return -e/(L*L)

def calc_m(s):
    s = s.reshape(L,L)
    m = 0
    for i in range(L):
        for j in range(L):
            m += s[i,j]
    return m/(L*L) 

In [3]:
import cython
%load_ext Cython

In [21]:
%%cython -a
                            # --compile-args="-O2"

cimport cython
import numpy as np

cimport numpy as cnp
from numpy cimport npy_intp, npy_longdouble
from libcpp cimport bool
from libc.math cimport exp


@cython.boundscheck(False)
cdef int calc_dE(npy_intp site,
                cnp.int32_t[::1] spins_1,  
                cnp.int32_t[::1] spins_2,
                cnp.int32_t[:, :] neighbors):    
    """Calculate e_jk: the energy change for spins[site] -> *= -1"""
    
    cdef:
        int site_1 = spins_1[site]
        int site_2 = spins_2[site]
        int dE = 0
        npy_intp idx
           
    
    for j in range(4):   
        idx = neighbors[site, j]
        dE += 2*site_1*(site_2*spins_1[idx]*spins_2[idx]+spins_1[idx])
    
    return dE


@cython.cdivision(True)
cdef bint mc_choice(int dE, double T, npy_longdouble uni):
    """принимаем или не принимаем переворот спина?"""
    cdef double r
    r = exp(-dE/T)
    if dE <= 0:
        return True
    elif uni <= r:
        return True
    else:
        return False

    

@cython.boundscheck(False) 
cdef void step(cnp.int32_t[::1] spins_s, cnp.int32_t[::1] spins_t,
               cnp.int32_t[:, :] neigh, double T, npy_intp lattice, npy_intp site, npy_longdouble uni):
    """крутим 1 спин"""
        
    cdef int L2, dE
    
    L2 = spins_s.shape[0]
    
    if lattice == 1:
        dE = calc_dE(site, spins_s, spins_t, neigh)   # if lattice s -> pass s,t
        if mc_choice(dE, T, uni):
            spins_s[site] *= -1
    else:
        dE = calc_dE(site, spins_t, spins_s, neigh)   # if lattice t -> pass t,s
        if mc_choice(dE, T, uni):
            spins_t[site] *= -1
    
    
        

@cython.boundscheck(False)
def mc_step(cnp.int32_t[::1] spins_s,
            cnp.int32_t[::1] spins_t,
            cnp.int32_t[:, :] neighbors,
            double T):
    """perform L*L flips for 1 MC step"""
    
    cdef npy_intp num_steps = spins_s.shape[0]
    cdef int _
    
    cdef cnp.ndarray[double,
                ndim=1,
                negative_indices=False,
                mode='c'] unis = np.random.uniform(size=num_steps)
    cdef cnp.ndarray[npy_intp,
                ndim=1,
                negative_indices=False,
                mode='c'] sites = np.random.randint(num_steps, size=num_steps)
    cdef cnp.ndarray[npy_intp,
                ndim=1,
                negative_indices=False,
                mode='c'] lattice = np.random.randint(2, size=num_steps)
    
    for _ in range(num_steps):
        step(spins_s, spins_t, neighbors, T, lattice[_], sites[_], unis[_])
       

@cython.boundscheck(False)  
# @cython.cdivision(True)
cdef double calc_e_c(cnp.int32_t[::1] spins_s,
                  cnp.int32_t[::1] spins_t,
                  cnp.int32_t[:, :] neighbors):
    cdef npy_intp L2 = spins_s.shape[0]
    cdef int site,j,idx
    cdef int E = 0
    cdef double r
    
    for site in range(L2):
        for j in range(2):
            idx = neighbors[site, j]
            E += (spins_s[site]*spins_s[idx] + spins_t[site]*spins_t[idx] + 
                  spins_s[site]*spins_s[idx]*spins_t[site]*spins_t[idx])
    r = -E/L2
    return r

def calc_e2(cnp.int32_t[::1] spins_s,
                  cnp.int32_t[::1] spins_t,
                  cnp.int32_t[:, :] neighbors):
    cdef double E
    E = calc_e_c(spins_s, spins_t, neighbors)
    return E


@cython.boundscheck(False)  
# @cython.cdivision(True)
cdef double calc_m_c(cnp.int32_t[::1] spins):
    cdef npy_intp L2 = spins.shape[0]
    cdef int site,j,idx
    cdef int M = 0
    cdef double r
    
    for site in range(L2):
        M += spins[site]
    r = M/L2
    return r

def calc_m2(cnp.int32_t[::1] spins):
    cdef double M
    M = calc_m_c(spins)
    return M

In [25]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); nei = get_neigh()#gen_e_dict()

%timeit [mc_step(s,t,nei,1) for _ in range(20)]

14.4 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); nei = get_neigh()#gen_e_dict()
print(calc_e2(s,t,nei))

for _ in range(20):
    mc_step(s,t, nei, 1)
print(calc_e2(s,t,nei))

-0.0556
-4.1844


In [ ]:
# py              -- 4.83 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# py + neigh      -- 13.6 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# cy              -- 14.9 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [23]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); nei = get_neigh()#gen_e_dict()

%timeit [calc_m(s) for _ in range(20)]

88.9 ms ± 4.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); nei = get_neigh()#gen_e_dict()

%timeit [calc_m2(s) for _ in range(20)]

184 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
